In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from scipy.sparse import hstack, csr_matrix
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import cross_val_score
import scipy.sparse as sparse
import string
import gc

from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn import preprocessing 

from sklearn.model_selection import GridSearchCV
import seaborn as sns
%matplotlib inline
sns.set(style="darkgrid")

In [3]:
df_train = pd.read_csv('./data/train.csv',encoding='utf-8')

In [4]:
df_html = pd.read_csv('./data/html_data.csv',encoding='utf-8')

In [5]:
df_test = pd.read_csv('./data/test.xls',encoding='utf-8')

In [6]:
len(df_train)

53447

In [7]:
len(df_test)

25787

In [8]:
len(df_html)

79345

In [13]:
df_train.head()

,Webpage_id,Domain,Url,Tag
0,1,www.fiercepharma.com,http://www.fiercepharma.com/marketing/tecfider...,news
1,2,www.fiercepharma.com,http://www.fiercepharma.com/pharma/novo-equipp...,news
2,3,www.fiercepharma.com,http://www.fiercepharma.com/pharma/another-exe...,news
3,4,www.fiercepharma.com,http://www.fiercepharma.com/pharma/teva-buy-bi...,news
4,5,www.fiercepharma.com,http://www.fiercepharma.com/marketing/actress-...,news


In [14]:
len(df_train['Domain'].unique())

3974

In [15]:
df_train['Domain'].value_counts()

thesis.library.caltech.edu               301
www.dart-europe.eu                       300
academiccommons.columbia.edu             300
dspace.mit.edu                           300
curate.nd.edu                            300
ecommons.cornell.edu                     300
www.nice.org.uk                          230
www.ncbi.nlm.nih.gov                     226
ctri.nic.in                              209
www.australiancancertrials.gov.au        209
rctportal.niph.go.jp                     206
slctr.lk                                 203
en.search.irct.ir                        201
cris.nih.go.kr                           200
www.clinicaltrials.jp                    200
www.anzctr.org.au                        200
www.trialregister.nl                     200
www.clinicaltrials.in.th                 200
www.medbox.org                           200
www.isrctn.com                           200
www.clinicalguidelines.gov.au            200
upload.umin.ac.jp                        200
www.chictr

In [16]:
len(df_test['Domain'].unique())

2150

In [ ]:
print(df_html.iloc[0][1])

In [17]:
from bs4 import BeautifulSoup
from bs4.element import Comment
import re


def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True


def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    #print(type(soup))
    texts = soup.findAll(text=True)
    #print(texts)
    visible_texts = filter(tag_visible, texts) 
    text = u" ".join(t.strip() for t in visible_texts)
    text = text.replace(',','')
    return (re.sub(r'\s\s+',' ',text))
text = (text_from_html(df_html.iloc[0][1]))

In [18]:
df_html['web_text'] = df_html['Html'].apply(lambda x: text_from_html(x))

In [19]:
del df_html['Html']

In [20]:
resulttrain = pd.merge(df_train, df_html, how='left', on=['Webpage_id'])

In [30]:
resulttrain.isnull().sum()

Webpage_id    0
Tag           0
web_text      0
url_length    0
url_text      0
dtype: int64

In [27]:
resulttrain['url_clean'] = resulttrain.apply(lambda x: str(x['Url']).replace(x['Domain'],''),axis=1)
del resulttrain['Domain']
resulttrain['url_clean'] = resulttrain['url_clean'].apply(lambda x: (re.split('https:///|http:///|/|-|=',x))[1:])
resulttrain['url_length'] = resulttrain['url_clean'].apply(lambda x: len(x))
resulttrain['url_text'] = resulttrain['url_clean'].apply(lambda x: ' '.join(x))
del resulttrain['Url']
del resulttrain['url_clean']

In [28]:
resulttrain.head()

,Webpage_id,Tag,web_text,url_length,url_text
0,1,news,Skip to main content Twitter LinkedIn Search ...,17,marketing tecfidera gilenya and aubagio s 3 wa...
1,2,news,Skip to main content Twitter LinkedIn Search ...,12,pharma novo equipped to weather storm u s diab...
2,3,news,Skip to main content Twitter LinkedIn Search ...,13,pharma another exec departs troubled endo and ...
3,4,news,Skip to main content Twitter LinkedIn Search ...,11,pharma teva buy biosim specialist celltrion it...
4,5,news,Skip to main content Twitter LinkedIn Search ...,12,marketing actress marissa tomei partners aller...


In [29]:
resulttrain.to_csv('trainwHDF.csv',encoding='utf-8')

In [31]:
resulttest = pd.merge(df_test, df_html, how='left', on=['Webpage_id'])

In [32]:
resulttest.isnull().sum()

Webpage_id    0
Domain        0
Url           0
web_text      0
dtype: int64

In [33]:
resulttest['url_clean'] = resulttest.apply(lambda x: str(x['Url']).replace(x['Domain'],''),axis=1)
del resulttest['Domain']
resulttest['url_clean'] = resulttest['url_clean'].apply(lambda x: (re.split('https:///|http:///|/|-|=',x))[1:])
resulttest['url_length'] = resulttest['url_clean'].apply(lambda x: len(x))
resulttest['url_text'] = resulttest['url_clean'].apply(lambda x: ' '.join(x))
del resulttest['Url']
del resulttest['url_clean']

In [34]:
resulttest.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25787 entries, 0 to 25786
Data columns (total 4 columns):
Webpage_id    25787 non-null int64
web_text      25787 non-null object
url_length    25787 non-null int64
url_text      25787 non-null object
dtypes: int64(2), object(2)
memory usage: 1007.3+ KB


In [90]:
resulttest[(resulttest['Webpage_id']<75584) | (resulttest['Webpage_id']>75594)].to_csv('testwHDF.csv',encoding='utf-8')

In [ ]:
df=resulttest[(resulttest['Webpage_id']>=75584) & (resulttest['Webpage_id']<=75594)].copy()

In [ ]:
df['web_text']=df['web_text'].apply(lambda x : x[2511:2520])

In [ ]:
print(df['web_text'])

In [ ]:
df=resulttest[(resulttest['Webpage_id']>=75584) & (resulttest['Webpage_id']<=75594)].copy()